In [1]:
! pip install pytorch_pretrained_bert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 123 kB 27.3 MB/s 
     |████████████████████████████████| 132 kB 49.8 MB/s 
     |████████████████████████████████| 9.3 MB 47.1 MB/s 
     |████████████████████████████████| 79 kB 6.2 MB/s 
     |████████████████████████████████| 140 kB 53.8 MB/s 
     |████████████████████████████████| 127 kB 59.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
import os

In [9]:
data_dir = "/content/drive/MyDrive/Colab Notebooks/Capstone/data/gweb_sancl"
answer_dir = os.path.join(data_dir, "pos_file", "answers")
wsj_dir = os.path.join(data_dir, "pos_file", "wsj")
labeled_dir = os.path.join(data_dir, "unlabeled")

In [8]:

train_file = os.path.join(wsj_dir, "gweb-wsj-train.conll")
dev_file = os.path.join(wsj_dir, "gweb-wsj-dev.conll")

In [ ]:
import codecs

In [ ]:
def read_conll_file(file_name, raw=False):
    """
    read in conll file
    word1    tag1
    ...      ...
    wordN    tagN
    Sentences MUST be separated by newlines!
    :param file_name: file to read in
    :param raw: if raw text file (with one sentence per line) -- adds 'DUMMY' label
    :return: generator of instances ((list of  words, list of tags) pairs)
    """
    current_words = []
    current_tags = []
    
    for line in codecs.open(file_name, encoding='utf-8'):
        #line = line.strip()
        line = line[:-1]

        if line:
            if raw:
                current_words = line.split() ## simple splitting by space
                current_tags = ['DUMMY' for _ in current_words]
                yield (current_words, current_tags)

            else:
                if len(line.split("\t")) != 2:
                    if len(line.split("\t")) == 1: # emtpy words in gimpel
                        raise IOError("Issue with input file - doesn't have a tag or token?")
                    else:
                        print("erroneous line: {} (line number: {}) ".format(line), file=sys.stderr)
                        exit()
                else:
                    word, tag = line.split('\t')
                current_words.append(word)
                current_tags.append(tag)

        else:
            if current_words and not raw: #skip emtpy lines
                yield (current_words, current_tags)
            current_words = []
            current_tags = []

    # check for last one
    if current_tags != [] and not raw:
        yield (current_words, current_tags)

In [ ]:
train_word_lst = []
train_tag_lst = []
tags = []
for word, tag in read_conll_file(train_file):
  train_word_lst.append(word)
  train_tag_lst.append(tag)
  tags.extend(tag)

dev_word_lst = []
dev_tag_lst = []
for word, tag in read_conll_file(dev_file):
  dev_word_lst.append(word)
  dev_tag_lst.append(tag)
  tags.extend(tag)

In [ ]:
len(train_word_lst)

30060

In [ ]:
len(dev_word_lst)

1336

Show sentences from training set and testing set

In [ ]:
" ".join(train_word_lst[0])

"In an Oct. 19 review of `` The Misanthrope '' at Chicago 's Goodman Theatre -LRB- `` Revitalized Classics Take the Stage in Windy City , '' Leisure & Arts -RRB- , the role of Celimene , played by Kim Cattrall , was mistakenly attributed to Christina Haag ."

In [ ]:
" ".join(train_tag_lst[0])

"IN DT NNP CD NN IN `` DT NN '' IN NNP POS NNP NNP -LRB- `` VBN NNS VBP DT NN IN NNP NNP , '' NN CC NNS -RRB- , DT NN IN NNP , VBN IN NNP NNP , VBD RB VBN IN NNP NNP ."

The total number of tags is 48, and we need to add \<pad\> as a tagging.

In [2]:
tags = list(set(tags))
tags = ["<pad>"] + tags

In [ ]:
len(tags)

49

In [ ]:
print(tags)

['<pad>', 'RBR', 'DT', '.', 'PRP$', 'SYM', 'CC', 'JJS', 'WRB', 'RP', 'NNPS', 'PDT', 'MD', 'FW', 'RBS', 'PRP', 'NNP', 'JJR', 'UH', "''", 'WP$', 'VBD', 'NFP', 'WP', ':', 'WDT', 'IN', 'POS', 'TO', 'NN', '-RRB-', '``', 'AFX', '$', 'HYPH', 'XX', ',', 'RB', '-LRB-', 'VBG', 'CD', 'NNS', 'JJ', 'VBZ', 'VB', 'EX', 'VBP', 'LS', 'VBN']


Build two dictionaries for tokens and taggings.

In [ ]:
tag2idx = {tag:idx for idx, tag in enumerate(tags)}
idx2tag = {idx:tag for idx, tag in enumerate(tags)}

In [ ]:
import os
from tqdm import tqdm_notebook as tqdm
import numpy as np
import torch
import torch.nn as nn
from torch.utils import data
import torch.optim as optim
from pytorch_pretrained_bert import BertTokenizer

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Data Loader

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)

100%|██████████| 213450/213450 [00:00<00:00, 835754.28B/s]


**This part is to help us understand the data processing.**

- For any sentence, we need to add \[CLS\] at the beginning and \[SEP\] at the end.
- The tagging of \[CLS\] and \[SEP\] is \<pad\>.

In [ ]:
sents = []
tags_li = []
for i in range(len(train_word_lst)):
  sents.append(["[CLS]"] + train_word_lst[i] + ["[SEP]"])
  tags_li.append(["<pad>"] + train_tag_lst[i] + ["<pad>"])

In [ ]:
words, tags = sents[0], tags_li[0]

Since the token that BERT used is different from what we have, we need to re-tokenize the sentence. Therefore, we may get different tokens. For those tokens, we only count the first piece of them.

- For example, the original token we had is **Oct.**, and the token processed by BERT is two pieces **Oct** and **.**
- Therefore, we use **is_head** to represent the tokens that we will use. The taggings with **1** is what we will use, and the taggings with **0** is what will not use

In [ ]:
# This part is to understand the data processing
for i in range(10):
  print("\n")
  print(i)
  w = words[i]
  t = tags[i]
  print("word:", w, "tag:", t)

  tokens = tokenizer.tokenize(w) if w not in ("[CLS]", "[SEP]") else [w]
  print("tokens:", tokens)
  xx = tokenizer.convert_tokens_to_ids(tokens)
  print("xx:", xx)

  is_head = [1] + [0]*(len(tokens) - 1)
  print("is_head:", is_head)

  t = [t] + ["<pad>"] * (len(tokens) - 1)  # <PAD>: no decision
  print("t:", t)
  yy = [tag2idx[each] for each in t]  # (T,)
  print("yy:", yy)




0
word: [CLS] tag: <pad>
tokens: ['[CLS]']
xx: [101]
is_head: [1]
t: ['<pad>']
yy: [0]


1
word: In tag: IN
tokens: ['In']
xx: [1130]
is_head: [1]
t: ['IN']
yy: [26]


2
word: an tag: DT
tokens: ['an']
xx: [1126]
is_head: [1]
t: ['DT']
yy: [2]


3
word: Oct. tag: NNP
tokens: ['Oct', '.']
xx: [14125, 119]
is_head: [1, 0]
t: ['NNP', '<pad>']
yy: [16, 0]


4
word: 19 tag: CD
tokens: ['19']
xx: [1627]
is_head: [1]
t: ['CD']
yy: [40]


5
word: review tag: NN
tokens: ['review']
xx: [3189]
is_head: [1]
t: ['NN']
yy: [29]


6
word: of tag: IN
tokens: ['of']
xx: [1104]
is_head: [1]
t: ['IN']
yy: [26]


7
word: `` tag: ``
tokens: ['`', '`']
xx: [169, 169]
is_head: [1, 0]
t: ['``', '<pad>']
yy: [31, 0]


8
word: The tag: DT
tokens: ['The']
xx: [1109]
is_head: [1]
t: ['DT']
yy: [2]


9
word: Misanthrope tag: NN
tokens: ['Mi', '##san', '##throp', '##e']
xx: [12107, 9995, 21850, 1162]
is_head: [1, 0, 0, 0]
t: ['NN', '<pad>', '<pad>', '<pad>']
yy: [29, 0, 0, 0]


In [ ]:
class PosDataset(data.Dataset):
    def __init__(self, word_lst, tag_lst):
        sents, tags_li = [], [] # list of lists
        for i in range(len(word_lst)):
            sents.append(["[CLS]"] + word_lst[i] + ["[SEP]"])
            tags_li.append(["<pad>"] + tag_lst[i] + ["<pad>"])
        self.sents, self.tags_li = sents, tags_li

    def __len__(self):
        return len(self.sents)

    def __getitem__(self, idx):
        words, tags = self.sents[idx], self.tags_li[idx] # words, tags: string list

        # We give credits only to the first piece.
        x, y = [], [] # list of ids
        is_heads = [] # list. 1: the token is the first piece of a word
        for w, t in zip(words, tags):
            tokens = tokenizer.tokenize(w) if w not in ("[CLS]", "[SEP]") else [w]
            xx = tokenizer.convert_tokens_to_ids(tokens)

            is_head = [1] + [0]*(len(tokens) - 1)

            t = [t] + ["<pad>"] * (len(tokens) - 1)  # <PAD>: no decision
            yy = [tag2idx[each] for each in t]  # (T,)

            x.extend(xx)
            is_heads.extend(is_head)
            y.extend(yy)

        assert len(x)==len(y)==len(is_heads), "len(x)={}, len(y)={}, len(is_heads)={}".format(len(x), len(y), len(is_heads))

        # seqlen
        seqlen = len(y)

        # to string
        words = " ".join(words)
        tags = " ".join(tags)
        return words, x, is_heads, tags, y, seqlen


In [ ]:
def pad(batch):
    '''Pads to the longest sample'''
    f = lambda x: [sample[x] for sample in batch]
    words = f(0)
    is_heads = f(2)
    tags = f(3)
    seqlens = f(-1)
    maxlen = np.array(seqlens).max()

    f = lambda x, seqlen: [sample[x] + [0] * (seqlen - len(sample[x])) for sample in batch] # 0: <pad>
    x = f(1, maxlen)
    y = f(-2, maxlen)


    f = torch.LongTensor

    return words, f(x), is_heads, tags, f(y), seqlens

# Model

In [ ]:
from pytorch_pretrained_bert import BertModel

In [ ]:
class Net(nn.Module):
    def __init__(self, vocab_size=None):
        super().__init__()
        self.bert = BertModel.from_pretrained('bert-base-cased')

        self.fc = nn.Linear(768, vocab_size)
        self.device = device

    def forward(self, x, y):
        '''
        x: (N, T). int64
        y: (N, T). int64
        '''
        x = x.to(device)
        y = y.to(device)
        
        if self.training:
            self.bert.train()
            encoded_layers, _ = self.bert(x)
            enc = encoded_layers[-1]
        else:
            self.bert.eval()
            with torch.no_grad():
                encoded_layers, _ = self.bert(x)
                enc = encoded_layers[-1]
        
        logits = self.fc(enc)
        y_hat = logits.argmax(-1)
        return logits, y, y_hat

# Train and evaluate

In [ ]:
def train(model, iterator, optimizer, criterion):
    model.train()
    for i, batch in enumerate(iterator):
        words, x, is_heads, tags, y, seqlens = batch
        _y = y # for monitoring
        optimizer.zero_grad()
        logits, y, _ = model(x, y) # logits: (N, T, VOCAB), y: (N, T)
        # print(logits.shape)
        # print(logits)
        # print(y.shape)
        # print(y)
        # break

        logits = logits.view(-1, logits.shape[-1]) # (N*T, VOCAB)
        y = y.view(-1)  # (N*T,)

        loss = criterion(logits, y)
        loss.backward()

        optimizer.step()

        if i%10==0: # monitoring
            print("step: {}, loss: {}".format(i, loss.item()))

In [ ]:
def eval(model, iterator):
    model.eval()

    Words, Is_heads, Tags, Y, Y_hat = [], [], [], [], []
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            words, x, is_heads, tags, y, seqlens = batch

            _, _, y_hat = model(x, y)  # y_hat: (N, T)

            Words.extend(words)
            Is_heads.extend(is_heads)
            Tags.extend(tags)
            Y.extend(y.numpy().tolist())
            Y_hat.extend(y_hat.cpu().numpy().tolist())

    ## gets results and save
    with open("result", 'w') as fout:
        for words, is_heads, tags, y_hat in zip(Words, Is_heads, Tags, Y_hat):
            y_hat = [hat for head, hat in zip(is_heads, y_hat) if head == 1]
            preds = [idx2tag[hat] for hat in y_hat]
            assert len(preds)==len(words.split())==len(tags.split())
            for w, t, p in zip(words.split()[1:-1], tags.split()[1:-1], preds[1:-1]):
                fout.write("{} {} {}\n".format(w, t, p))
            fout.write("\n")
            
    ## calc metric
    y_true =  np.array([tag2idx[line.split()[1]] for line in open('result', 'r').read().splitlines() if len(line) > 0])
    y_pred =  np.array([tag2idx[line.split()[2]] for line in open('result', 'r').read().splitlines() if len(line) > 0])

    acc = (y_true==y_pred).astype(np.int32).sum() / len(y_true)

    print("acc=%.2f"%acc)


# Load model and train

In [ ]:
model = Net(vocab_size=len(tag2idx))
model.to(device)
model = nn.DataParallel(model)

100%|██████████| 404400730/404400730 [00:14<00:00, 28113427.12B/s]


In [ ]:
train_dataset = PosDataset(train_word_lst, train_tag_lst)
eval_dataset = PosDataset(dev_word_lst, dev_tag_lst)

train_iter = data.DataLoader(dataset=train_dataset,
                             batch_size=8,
                             shuffle=True,
                             num_workers=1,
                             collate_fn=pad)
test_iter = data.DataLoader(dataset=eval_dataset,
                             batch_size=8,
                             shuffle=False,
                             num_workers=1,
                             collate_fn=pad)

optimizer = optim.Adam(model.parameters(), lr = 0.0001)

criterion = nn.CrossEntropyLoss(ignore_index=0)

**This part is to help me understand the processed data and the model structure**

In [ ]:
for i, batch_iter in enumerate(train_iter):
  break

In [ ]:
for i in range(len(batch_iter)):
  print(f"batch_iter[{i}][0]:", batch_iter[i][0])

batch_iter[0][0]: [CLS] The $ 125 - billion - a - year Bay area economy represents one - fourth of the economy of the nation 's most populous state and accounts for 2 % to 3 % of the nation 's total output of goods and services , according to the Center for Continuing Study of the California Economy in Palo Alto . [SEP]
batch_iter[1][0]: tensor([  101,  1109,   109,  8347,   118,  3775,   118,   170,   118,  1214,
         2410,  1298,  4190,  5149,  1141,   118,  2223,  1104,  1103,  4190,
         1104,  1103,  3790,   112,   188,  1211, 22608,  1352,  1105,  5756,
         1111,   123,   110,  1106,   124,   110,  1104,  1103,  3790,   112,
          188,  1703,  5964,  1104,  4817,  1105,  1826,   117,  2452,  1106,
         1103,  1945,  1111, 20699,  8690,  1104,  1103,  1756, 14592,  1107,
        19585,  2858, 17762,   119,   102])
batch_iter[2][0]: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 

In [ ]:
for i in range(len(batch_iter)):
  print(f"batch_iter[{i}][1]:", batch_iter[i][1])

batch_iter[0][1]: [CLS] So far as we can see only two persons are behaving with a dignity recognizing the seriousness of the issues : Mr. Lawson and Sir Alan Walters , the counterpoint of the Chancellor 's difficulties , who also resigned as personal adviser to Mrs. Thatcher . [SEP]
batch_iter[1][1]: tensor([  101,  1573,  1677,  1112,  1195,  1169,  1267,  1178,  1160,  4983,
         1132,  1129, 22300,  1114,   170, 14931, 17344,  1103,  3021,  1757,
         1104,  1103,  2492,   131,  1828,   119, 15205,  1105,  2203,  4258,
        20789,   117,  1103,  4073,  7587,  1104,  1103,  8861,   112,   188,
         7866,   117,  1150,  1145,  4603,  1112,  2357, 14269,  1106,  2823,
          119, 23300,   119,   102,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0])
batch_iter[2][1]: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1]
batch

In [ ]:
x = batch_iter[1]
y = batch_iter[-2]
logits, y, _ = model(x, y)

In [ ]:
# Output: 8 sentences * 58 tokens * 49 taggings
logits.shape

torch.Size([8, 65, 49])

In [ ]:
logits[0, 0, :]

tensor([ 0.8223, -0.0341,  0.2307,  0.3083,  0.2271, -0.4868, -0.5868, -0.3462,
         0.3355, -0.1309, -0.3546,  0.5737, -0.0997, -0.1102, -0.8688, -0.1514,
         0.1021, -0.3319, -0.0531,  0.5775,  0.1398,  0.1345,  0.1572,  0.1084,
        -0.5019, -0.1154,  0.1645, -0.1370, -0.0526, -0.1087, -0.1527,  0.0240,
        -0.5115,  0.1530, -0.1388, -0.5213, -0.0163, -0.0054,  0.3151, -0.1519,
        -0.5824, -0.0512, -0.1611,  0.2870, -0.0377,  0.4715, -0.6076, -0.2004,
         0.2832], device='cuda:0', grad_fn=<SliceBackward0>)

In [ ]:
logits = logits.view(-1, logits.shape[-1]) # (N*T, VOCAB)
y = y.view(-1)  # (N*T,)

In [ ]:
logits.shape

torch.Size([520, 49])

In [ ]:
y.shape

torch.Size([520])

In [ ]:
logits[:, 1].shape

torch.Size([520])

In [ ]:
loss = criterion(logits, y)

In [ ]:
loss

tensor(3.8318, device='cuda:0', grad_fn=<NllLossBackward0>)

In [ ]:
train(model, train_iter, optimizer, criterion)

step: 0, loss: 3.8127729892730713
step: 10, loss: 1.9694442749023438
step: 20, loss: 0.879882276058197
step: 30, loss: 0.30797073245048523
step: 40, loss: 0.3897354304790497
step: 50, loss: 0.2547544538974762
step: 60, loss: 0.14905545115470886
step: 70, loss: 0.18233917653560638
step: 80, loss: 0.1623125970363617
step: 90, loss: 0.1022351086139679
step: 100, loss: 0.22179630398750305
step: 110, loss: 0.0652894452214241
step: 120, loss: 0.1724962443113327
step: 130, loss: 0.18637429177761078
step: 140, loss: 0.22088506817817688
step: 150, loss: 0.25937268137931824
step: 160, loss: 0.18992236256599426
step: 170, loss: 0.1468413919210434
step: 180, loss: 0.10014870017766953
step: 190, loss: 0.14707069098949432
step: 200, loss: 0.12163648009300232
step: 210, loss: 0.166069895029068
step: 220, loss: 0.07133134454488754
step: 230, loss: 0.05981002002954483
step: 240, loss: 0.16793608665466309
step: 250, loss: 0.1492682546377182
step: 260, loss: 0.10050888359546661
step: 270, loss: 0.1977836

In [ ]:
eval(model, test_iter)

acc=0.97


In [ ]:
open('result', 'r').read().splitlines()[:100]

['Influential JJ JJ',
 'members NNS NNS',
 'of IN IN',
 'the DT DT',
 'House NNP NNP',
 'Ways NNP NNPS',
 'and CC CC',
 'Means NNP NNPS',
 'Committee NNP NNP',
 'introduced VBD VBD',
 'legislation NN NN',
 'that WDT WDT',
 'would MD MD',
 'restrict VB VB',
 'how WRB WRB',
 'the DT DT',
 'new JJ JJ',
 'savings NNS NNS',
 '- HYPH HYPH',
 'and CC CC',
 '- HYPH HYPH',
 'loan NN NN',
 'bailout NN NN',
 'agency NN NN',
 'can MD MD',
 'raise VB VB',
 'capital NN NN',
 ', , ,',
 'creating VBG VBG',
 'another DT DT',
 'potential JJ JJ',
 'obstacle NN NN',
 'to IN IN',
 'the DT DT',
 'government NN NN',
 "'s POS POS",
 'sale NN NN',
 'of IN IN',
 'sick JJ JJ',
 'thrifts NNS NNS',
 '. . .',
 '',
 'The DT DT',
 'bill NN NN',
 ', , ,',
 'whose WP$ WP$',
 'backers NNS NNS',
 'include VBP VBP',
 'Chairman NNP NNP',
 'Dan NNP NNP',
 'Rostenkowski NNP NNP',
 '-LRB- -LRB- -LRB-',
 'D. NNP NNP',
 ', , ,',
 'Ill. NNP NNP',
 '-RRB- -RRB- -RRB-',
 ', , ,',
 'would MD MD',
 'prevent VB VB',
 'the DT DT',
 'R

# Other metrics

In [ ]:
train_dataset = PosDataset(train_word_lst, train_tag_lst)
eval_dataset = PosDataset(dev_word_lst, dev_tag_lst)

train_iter = data.DataLoader(dataset=train_dataset,
                             batch_size=8,
                             shuffle=True,
                             num_workers=1,
                             collate_fn=pad)
test_iter = data.DataLoader(dataset=eval_dataset,
                             batch_size=8,
                             shuffle=False,
                             num_workers=1,
                             collate_fn=pad)

optimizer = optim.Adam(model.parameters(), lr = 0.0001)

criterion = nn.CrossEntropyLoss(ignore_index=0)

In [ ]:
model.eval()

Words, Is_heads, Tags, Y, Y_hat = [], [], [], [], []
with torch.no_grad():
    for i, batch in enumerate(test_iter):
        words, x, is_heads, tags, y, seqlens = batch

        _, _, y_hat = model(x, y)  # y_hat: (N, T)

        Words.extend(words)
        Is_heads.extend(is_heads)
        Tags.extend(tags)
        Y.extend(y.numpy().tolist())
        Y_hat.extend(y_hat.cpu().numpy().tolist())

In [ ]:
len(dev_word_lst)

1336

In [ ]:
len(Y_hat)

1336

In [ ]:
y_hat.shape

torch.Size([8, 63])

In [ ]:
y_true =  np.array([tag2idx[line.split()[1]] for line in open('result', 'r').read().splitlines() if len(line) > 0])
y_pred =  np.array([tag2idx[line.split()[2]] for line in open('result', 'r').read().splitlines() if len(line) > 0])

In [ ]:
y_true.shape

(32092,)

In [ ]:
y_pred.shape

(32092,)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
## calc metric
y_true =  np.array([tag2idx[line.split()[1]] for line in open('result', 'r').read().splitlines() if len(line) > 0])
y_pred =  np.array([tag2idx[line.split()[2]] for line in open('result', 'r').read().splitlines() if len(line) > 0])

acc = (y_true==y_pred).astype(np.int32).sum() / len(y_true)

print("acc=%.2f"%acc)

acc=0.97


In [ ]:
# Calculate metrics globally by counting the total true positives, false negatives and false positives.
print("micro precision", precision_score(y_true, y_pred, average="micro"))
# Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account.
print("macro precision", precision_score(y_true, y_pred, average="macro"))
# Calculate metrics for each label, and find their average weighted by support (the number of true instances for each label). 
# This alters ‘macro’ to account for label imbalance; it can result in an F-score that is not between precision and recall.
print("weighted precision", precision_score(y_true, y_pred, average="weighted"))

micro precision 0.9731085628817151
macro precision 0.9073238140324092
weighted precision 0.9743623300289095


In [ ]:
print("micro recall", recall_score(y_true, y_pred, average="micro"))
print("macro recall", recall_score(y_true, y_pred, average="macro"))
print("weighted recall", recall_score(y_true, y_pred, average="weighted"))

micro recall 0.9731085628817151
macro recall 0.9151197467279802
weighted recall 0.9731085628817151


In [ ]:
print("micro f1", f1_score(y_true, y_pred, average="micro"))
print("macro f1", f1_score(y_true, y_pred, average="macro"))
print("weighted f1", f1_score(y_true, y_pred, average="weighted"))

micro f1 0.9731085628817152
macro f1 0.9043129765781763
weighted f1 0.973577776244746
